## Creando inputs para el modelo AMPL a partir de archivo csv: Archivo .dat

En este cuadernillo se desarrolla el código para insertar los distintos parámetros al modelo. En vista de su practicidad, se decidió insertar los datos usando objetos de pandas creados con un archivo csv

In [1]:
# Leemos los conjuntos
import pandas as pd
import numpy as np
import sys
import os


Origin = pd.DataFrame([
    ('manizales'),
    ('medellin'),
    ('bucaramanga'),
    ('bogota'),
    ('ibague'),
    ('neival'),
    ('soacha'),
    ('pereira'),
    ],
    columns=["Origin"],)#.set_index("Origin")

Destination = pd.DataFrame([
    ('españa'),
    ('alemania'),
    ('brazil'),
    ('panama'),
    ('mexico'),
    ('estados_unidos'),
    ],
    columns=["Destination"],)#.set_index("Destination")


CTL = pd.DataFrame([
    ('ctl_bogota'),
    ('ctl_pereira'),
    ('ctl_manizales'),
    ('ctl_soacha'),
    ('ctl_neiva'),
    ('ctl_ibague'),
    ('ctl_cienaga'),
    ('ctl_la_dorada'),
    ('ctl_puerto_berrio'),
    ],
    columns=["CTL"],)#.set_index("CTL")

PCTL = pd.DataFrame([
    ('ctl_soeldad'),
    ('ctl_santa_marta'),
    ('ctl_cartagena'),
    ],
    columns=["PCTL"],)#.set_index("PCTL")

Port = pd.DataFrame([
    ('puerto_de_cartagena'),
    ('puerto_de_barranquilla'),
    ('puerto_de_santa_marta'),
    ],
    columns=["Port"],)#.set_index("Port")

CAPk = pd.read_excel("model_data.xlsx",sheet_name="CAPk").set_index("CTL")
CAPm = pd.read_excel("model_data.xlsx",sheet_name="CAPm").set_index("PCTL")

Fk = pd.read_excel("model_data.xlsx",sheet_name="Fk").set_index("CTL")
Fm = pd.read_excel("model_data.xlsx",sheet_name="Fm").set_index("PCTL")

Wij=pd.read_excel("model_data.xlsx",sheet_name="Wij").set_index(['Origin', 'Destination'])

costs=pd.read_excel("model_data.xlsx",sheet_name="cost")
costs.set_index(['code_inicio', 'code_fin'], inplace=True)




In [2]:
# Origin 
# Destination
# CTL 
# PCTL 
# Port
# CAPk 
# CAPm
# Fk
# Fm
# Wij
# costs
valor = costs.loc[('puerto_de_cartagena', 'mexico'), 'transport_cost']

#Cijp
#Cijkp
#Cijmp
#Cijkmp

#Eijp
#Eijkp
#Eijmp
#Eijkmp
#Wij 
0.025*1000

/home/storreglosa/anaconda3/envs/multimodal_logistics_network_colombia/lib/python3.7/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


25.0

In [3]:
O=list(Origin['Origin'])
D=list(Destination['Destination'])
ks=list(CTL['CTL'])
ms=list(PCTL['PCTL'])
puer=list(Port['Port'])


#Cijp
Cijp=[]
for i in O:
    for j in D:
        for p in puer:
            try:
                valor=costs.loc[(i, p), 'transport_cost'] + costs.loc[(i, j), 'transport_cost']
                diccionario = {"i":i,"j":j,"p":p,"Cijp":valor}
                Cijp.append(diccionario)
            except:
                valor = np.inf
                diccionario = {"i":i,"j":j,"p":p,"Cijp":valor}
                Cijp.append(diccionario)
Cijp =pd.DataFrame(Cijp)
#Cijkp
Cijkp=[]
for i in O:
    for j in D:
        for k in ks:
            for p in puer:
                try:
                    valor=costs.loc[(i,k), 'transport_cost'] + costs.loc[(k,p), 'transport_cost'] + costs.loc[(p,j), 'transport_cost']
                    diccionario = {"i":i,"j":j,"k":k,"p":p,"Cijp":valor}
                    Cijkp.append(diccionario)
                    #print(valor)
                except:
                    valor = np.inf
                    #print(valor)
                    diccionario = {"i":i,"j":j,"k":k,"p":p,"Cijkp":valor}
                    Cijkp.append(diccionario)
Cijkp =pd.DataFrame(Cijkp)

#Cijmp
Cijmp=[]
for i in O:
    for j in D:
        for m in ms:
            for p in puer:
                try:
                    valor=costs.loc[(i,m), 'transport_cost'] + costs.loc[(m,p), 'transport_cost'] + costs.loc[(p,j), 'transport_cost']
                    diccionario = {"i":i,"j":j,"m":m,"p":p,"Cijmp":valor}
                    Cijmp.append(diccionario)
                    #print(valor)
                except:
                    valor = np.inf
                    #print(valor)
                    diccionario = {"i":i,"j":j,"m":m,"p":p,"Cijmp":valor}
                    Cijmp.append(diccionario)
Cijmp =pd.DataFrame(Cijmp)

#a=pd.Series(Cijmp["Cijmp"].unique())
#Cijkmp
#Eijp
#Eijkp
#Eijmp
#Eijkmp
#Wij
